In [5]:
# 環境設置
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys  # 鍵盤操作
from selenium.common.exceptions import NoSuchElementException
# ---------- 等待頁面加載完成
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
# ----------
import pandas as pd
import pandas.io.formats.excel
import requests
import requests.packages.urllib3
requests.packages.urllib3.disable_warnings()
from urllib.request import HTTPError
from bs4 import BeautifulSoup
import time

In [7]:
ISBN = "9789573317241"
title = "哈利波特(1)：神秘的魔法石"

df_lst = []

my_options = Options()
my_options.add_argument("--incognito")  # 開啟無痕模式
# # my_options.add_argument("--headless")  # 不開啟實體瀏覽器
driver = webdriver.Chrome(options=my_options)

In [ ]:
def easy_crawler(position, org, url, ISBN):
    try:
        # 組合成書的網址
        url = url + ISBN
        # 載入 html，如果發生 HTTPError，那麼就使用 requests.get(url, verify=False)
        try:
            dfs = pd.read_html(url, encoding="utf-8")
        except HTTPError:
            resp = requests.get(url, verify=False)  # 設定 verify=False，以解決 SSLError
            dfs = pd.read_html(resp.text, encoding="utf-8")
        # 定位表格
        tgt = dfs[position]
        
        # tgt.insert(0, "連結", [url for i in range(tgt.shape[0])])
        # tgt.insert(0, "圖書館", [org for i in range(tgt.shape[0])])
        return tgt
    except:
        print(f"「{url}」無法爬取！")

In [ ]:
dfs = []

dfs.append(crawler_only_pandas(2, "國立臺灣海洋大學", "https://ocean.ntou.edu.tw/search*cht/i?SEARCH="))
dfs.append(crawler_only_pandas(4, "國立臺灣師範大學", "https://opac.lib.ntnu.edu.tw/search*cht/i?SEARCH="))
dfs.append(crawler_only_pandas(4, "中央研究院", "https://las.sinica.edu.tw/search*cht/a?searchtype=i&searcharg="))
dfs.append(crawler_only_pandas(4, "國立陽明大學", "https://library.ym.edu.tw/search*cht/a?searchtype=i&searcharg="))
dfs.append(crawler_only_pandas(6, "國立中央大學", "https://opac.lib.ncu.edu.tw/search*cht/i?SEARCH="))
dfs.append(crawler_only_pandas(6, "中原大學", "https://cylis.lib.cycu.edu.tw/search*cht/~?searchtype=i&searcharg=")) # 中原大學：同一個 ISBN 會找到紙本書和電子書
dfs.append(crawler_only_pandas(6, "國立臺灣科技大學", "https://sierra.lib.ntust.edu.tw/search*cht/i?SEARCH="))
dfs.append(crawler_only_pandas(7, "中國文化大學", "https://webpac.pccu.edu.tw/search*cht/?searchtype=i&searcharg="))
dfs.append(crawler_only_pandas(7, "輔仁大學", "https://library.lib.fju.edu.tw/search~S0*cht/?searchtype=i&searcharg="))

In [ ]:
df_final = pd.concat(dfs, axis=0, ignore_index=True)
df_final.fillna("", inplace=True)
df_final["索書號"] = df_final["索書號/期刊合訂本卷期"] + df_final["索書號"] + df_final["索書號 / 部冊號"]
df_final["處理狀態"] = df_final["處理狀態"] + df_final["館藏狀況(月-日-西元年)"] + df_final["狀態 (說明)"] + df_final["館藏現況 說明"]
df_final.drop(["條碼", "OPAC 訊息", "其它訊息", "冊次", "年代", "條碼號"], axis=1, inplace=True)
df_final.drop(["索書號/期刊合訂本卷期", "索書號 / 部冊號"], axis=1, inplace=True)
df_final.drop(["館藏狀況(月-日-西元年)", "狀態 (說明)", "館藏現況 說明"], axis=1, inplace=True)
df_final["館藏地"] = df_final["館藏地"].str.replace("斈", "學")  # 解決輔仁大學的「斈」字所導致的 UnicodeEncodeError

In [ ]:
df_final

In [ ]:
# 解決 UnicodeEncodeError
# df_final["館藏地"] = df_final["館藏地"].str.split().str.join(" ")
# df_final["索書號"] = df_final["索書號"].str.split().str.join(" ")
# df_final["處理狀態"] = df_final["處理狀態"].str.split().str.join(" ")

In [ ]:
# 輸出成 csv 格式
# df_final.to_csv("perfectData.csv", encoding="big5", index=True)

In [ ]:
# 輸出成 excel
# df_final.to_excel(f"《{title}》的搜尋結果2.xlsx", encoding="big5", index=True)

In [8]:
# 世新大學
try:
    url = "https://lib.shu.edu.tw/"
    driver.get(url)

    # 定位選擇欄，以 ISBN 搜尋方式搜尋
    select = Select(driver.find_element_by_name("idx"))
    select.select_by_visible_text(u"ISBN")
    search_input = driver.find_element_by_name("q")
    search_input.send_keys(ISBN)
    search_input.submit()

    # 遇到「你要提交的資訊未受到保護」，點選仍要傳送
    continue_btn = driver.find_element_by_xpath('//*[@id=\"proceed-button\"]')
    continue_btn.click()

    url = driver.current_url
    html_text = driver.page_source
    dfs = pd.read_html(html_text, encoding="utf-8")
    tgt = dfs[0]

#     tgt.insert(0, "圖書館", ["世新大學" for i in range(tgt.shape[0])])
#     for deleted in ["館藏", "到期日", "條碼", "館藏預約"]:
#         tgt.pop(deleted)
#     tgt.rename(columns={"現行位置": "館藏地",
#                            "狀態": "館藏狀態"}, inplace=True)
#     tgt["館藏地"] = tgt["館藏地"].str.replace("世新大學圖書館", "")
    df_lst.append(tgt)
except:
    print(f"「{url}」無法爬取！")
tgt

,現行位置,館藏,索書號,狀態,到期日,條碼,館藏預約
0,世新大學圖書館 中文圖書區,圖書,873.57 857 c.5 (瀏覽書架),借出,2021-09-30,C405302,NaN
1,世新大學圖書館 中文圖書區,圖書,873.57 857 c.4 (瀏覽書架),可借,NaN,C306599,NaN
2,世新大學圖書館 中文圖書區,圖書,873.57 857 c.3 (瀏覽書架),借出,2021-10-01,C267795,NaN
3,世新大學圖書館 中文圖書區,圖書,873.57 857 (瀏覽書架),可借,NaN,C185698,NaN
4,世新大學圖書館 中文圖書區,圖書,873.57 857 c.2 (瀏覽書架),可借,NaN,C185699,NaN


In [ ]:
# 國立臺北大學
try:
    url = f"http://webpac.lib.ntpu.edu.tw/search.cfm?m=ss&k0={ISBN}&t0=k&c0=and"
    driver.get(url)

    html_text = driver.page_source
    soup = BeautifulSoup(html_text, "html.parser")

    # 會搜尋到多本書，需要取得每本書的網址
    hrefs = []
    list_boxes = soup.find_all("div", "list_box")
    for list_box in list_boxes:
        hrefs.append("http://webpac.lib.ntpu.edu.tw/" + list_box.find("a", "btn")["href"])

    # 開始爬表格
    tgt = pd.DataFrame()
    for url in hrefs:
        # 進入頁面爬取表格
        dfs = pd.read_html(url, encoding="utf-8")
        df_temp = dfs[0]

        # 新增欄位《館藏地》
        resp = requests.get(url)
        soup = BeautifulSoup(resp.text, "html.parser")
        place_dad = soup.find("div", "book_area").find("h2").text
        place_son = soup.find("div", "book_area").find("h3").text
        df_temp.insert(1, "館藏地", [place_dad + place_son for i in range(df_temp.shape[0])])

        # 合併表格
        tgt = pd.concat([tgt, df_temp], axis=0, ignore_index=True)

    # 新增欄位《圖書館》
    tgt.insert(0, "圖書館", ["國立臺北大學" for i in range(tgt.shape[0])])
    # 刪除不要的欄位
    for deleted in ["登錄號", "資料類型", "使用類型", "附件", "預約人數", "Unnamed: 7"]:
        tgt.pop(deleted)
    df_lst.append(tgt)
except:
    print(f"「{url}」無法爬取！")
tgt

In [ ]:
# 國立臺灣博物館
try:
    # ISBN = 9789865321703
    url = "https://lib.moc.gov.tw/F"
    driver.get(url)

    time.sleep(1)  # 為了等待網頁加載

    select = Select(driver.find_element_by_name("x"))
    select.select_by_visible_text(u"ISBN")
    search_input = driver.find_element_by_name("y")
    search_input.send_keys(ISBN)
    # search_input.submit()  # 不知道為什麼無法 submit()？
    submit_input = driver.find_element_by_name("Search")
    submit_input.click()

    click = driver.find_element_by_xpath("/html/body/table[9]/tbody/tr/td[1]/table/tbody/tr[1]/td[2]/a")
    click.click()

    html_text = driver.page_source
    dfs = pd.read_html(html_text, encoding="utf-8")
    tgt = dfs[11]

    # 清理資料
    tgt = tgt.drop(tgt.columns[[0, 1, 5, 6, 8, 9, 10]], axis=1)
    # 處理有 JS 語法問題
    for i in range(tgt.shape[0]):
        tgt.at[i, "到期日"] = tgt.at[i, "到期日"].split(";")[-1].strip()

    tgt.rename(columns={"館藏地": "圖書館",
                            "典藏區域": "館藏地",
                            "到期日": "館藏狀態"}, inplace=True)
    tgt["圖書館"] = tgt["圖書館"].str.replace("臺史博", "國立臺灣博物館")
    df_lst.append(tgt)
except:
    print(f"「{url}」無法爬取！")
tgt

In [ ]:
# 國立臺北護理健康大學
try:
    # 搜尋
    url = "http://140.131.94.8/uhtbin/webcat/"
    driver.get(url)

    select = Select(driver.find_element_by_name("srchfield1"))
    select.select_by_visible_text(u"所有欄位")
    search_input = driver.find_element_by_name("searchdata1")
    search_input.send_keys(ISBN)
    search_input.submit()

    # 取得 dataframe
    html_text = driver.page_source
    dfs = pd.read_html(html_text, encoding="utf-8")
    tgt = dfs[0]

    # 整理 dataframe
    place = tgt.at[1, 0]
    tgt.drop([0, 1, 2], inplace=True)
    tgt.drop([1, 2, 4], axis=1, inplace=True)
    tgt.insert(0, "圖書館", ["國立臺北護理健康大學" for i in range(tgt.shape[0])])
    tgt.insert(1, "館藏地", [place for i in range(tgt.shape[0])])
    tgt.rename(columns={0: "索書號",
                              3: "館藏狀態"}, inplace=True)
    tgt.fillna(method="ffill", axis=0, inplace=True)  # 遇到值為 NaN時，將前一列的值填補進來
    tgt.reset_index(inplace=True, drop=True)  # 重置索引值
    df_lst.append(tgt)
except:
    print(f"「{url}」無法爬取！")
tgt

In [ ]:
# 國立清華大學
try:
    url = "https://webpac.lib.nthu.edu.tw/F/"
    driver.get(url)
    
    print("等待頁面加載中…")
    for i in range(1, 4):
        time.sleep(1)
        print(i)
    
    select = Select(driver.find_element_by_name("x"))
    select.select_by_visible_text(u"ISSN / ISBN")
    search_input = driver.find_element_by_name("y")
    search_input.send_keys(ISBN)
    # search_input.submit()  # 不知道為什麼無法 submit()？
    submit_input = driver.find_element_by_name("Search")
    submit_input.click()
    driver.find_element_by_xpath("//*[@id=\"gridview\"]/tbody/tr[2]/td[3]/a").click()
    driver.find_element_by_xpath("//*[@id=\"館藏地\"]/a").click()
    
    # 取得 dataframe
    html_text = driver.page_source
    dfs = pd.read_html(html_text, encoding="utf-8")
    tgt = dfs[13]

    # 整理 dataframe
    tgt.drop(tgt.columns[[0, 1, 2, 5, 7, 9, 10, 11, 12]], axis=1, inplace=True)
    tgt["館藏地"] = tgt["館藏地"].str.split("; }").str[-1].str.replace("/", "")
    tgt["卷期描述"].fillna("", inplace=True)
    tgt["索書號"] = tgt["索書號"].str.cat(tgt["卷期描述"], sep=" ")
    tgt.pop("卷期描述")
    tgt["到期日"] = tgt["到期日"].str.split(";").str[-1]
    tgt.insert(0, "圖書館", ["國立清華大學" for i in range(tgt.shape[0])])
    tgt.rename(columns={"到期日": "館藏狀態"}, inplace=True)
    df_lst.append(tgt)
except:
    print(f"「{url}」無法爬取！")
tgt

In [ ]:
# 將所有的 DataFrame 垂直合併成一個 DataFrame
df_final = pd.concat(df_lst, axis=0, ignore_index=True)

df_final["館藏地"] = df_final["館藏地"]\
.str.strip()

df_final["索書號"] = df_final["索書號"]\
.str.replace("《鄰近架位館藏》", "")\
.str.replace("\(瀏覽書架\)", "", regex=True)\
.str.strip()

df_final["館藏狀態"] = df_final["館藏狀態"]
# .str.replace("到期 \d\d-\d\d-\d\d", "外借中", regex=True)\
# .str.replace("到期 \d\d/\d\d/\d\d", "外借中", regex=True)\
# .str.replace("在架上", "可外借")\
# .str.replace("仍在館內 Available", "可外借", regex=True)\
# .str.strip()
# .str.replace("已被外借 On loan 到期日 \d\d\d\d-\d\d-\d\d", "不可外借", regex=True)\
# .str.replace("\+1 已催還", "", regex=True)\

df_final

In [ ]:
# 解決 UnicodeEncodeError
df_final["館藏地"] = df_final["館藏地"].str.split().str.join(" ")
df_final["索書號"] = df_final["索書號"].str.split().str.join(" ")
df_final["館藏狀態"] = df_final["館藏狀態"].str.split().str.join(" ")

In [ ]:
# 輸出成 csv 格式
df_final.to_csv(f"{title}的搜尋結果.csv", encoding="big5", index=True)